In [1]:
from dotenv import load_dotenv
import os

env_path = "../OllamaSetup/.env"  # Relative path from PromptTemplates
load_dotenv(env_path)

# Check if variables are loaded
print(os.getenv("LANGCHAIN_ENDPOINT")) 

https://api.smith.langchain.com


## Sequential LCEL

In [2]:
from langchain_ollama import ChatOllama
base_url="http://localhost:11434"
model = 'llama3.2:1b'
# model='sheldon'


llm = ChatOllama(
    base_url=base_url,
    model = model)

In [ ]:
from langchain_core.prompts import (
                                        SystemMessagePromptTemplate,
                                        HumanMessagePromptTemplate,
                                        ChatPromptTemplate
)


In [9]:
question = HumanMessagePromptTemplate.from_template("Tell me about {topic} in {points} points")
sys_message = SystemMessagePromptTemplate.from_template("You are {role}.You answer in short sentences")

input_msg=[sys_message,question]
input_template= ChatPromptTemplate(input_msg)

final_question = input_template.invoke({'topic':'Planets','points':'5','role':'teacher'})
final_question
ans= llm.invoke(final_question)
print(ans.content)


Here are five key points about planets:

1. There are eight planets in our solar system, divided into two categories: the inner planets (Mercury to Venus) and the outer planets (Neptune to Uranus).
2. The eight planets in our solar system are: Mercury, Mars, Earth, Jupiter, Saturn, Uranus, Neptune, and Pluto.
3. Our Sun is the star at the center of our solar system, and it gives us light and heat for life on Earth.
4. The four inner planets (Mercury to Venus) are very hot during the day and get much colder at night due to their close proximity to the Sun.
5. The outer planets (Jupiter to Neptune) are icy and have very thin atmospheres, with Jupiter being the largest planet in our solar system.


### using chain

In [10]:
question = HumanMessagePromptTemplate.from_template("Tell me about {topic} in {points} points")
sys_message = SystemMessagePromptTemplate.from_template("You are {role}.You answer in short sentences")

input_msg=[sys_message,question]
input_template= ChatPromptTemplate(input_msg)

In [11]:
input_chain= input_template | llm
input_chain

ChatPromptTemplate(input_variables=['points', 'role', 'topic'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['role'], input_types={}, partial_variables={}, template='You are {role}.You answer in short sentences'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['points', 'topic'], input_types={}, partial_variables={}, template='Tell me about {topic} in {points} points'), additional_kwargs={})])
| ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434')

In [13]:
ans= input_chain.invoke({'topic':'Planets','points':'5','role':'teacher'})
print(ans.content)

Here's what I know about planets:

1. The Sun is the center of our solar system, and it makes up 99% of its mass.
2. There are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.
3. Planets can be classified into two types: terrestrial (rocky) planets like Earth and Mars, and gas giants like Jupiter and Saturn.
4. The largest planet is Jupiter, which has a diameter of over 89,000 miles. The smallest planet is Mercury, which has a diameter of about 4,879 miles.
5. Each planet has its own unique features, such as moons, rings, and atmospheres.


In [14]:
ans

AIMessage(content="Here's what I know about planets:\n\n1. The Sun is the center of our solar system, and it makes up 99% of its mass.\n2. There are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.\n3. Planets can be classified into two types: terrestrial (rocky) planets like Earth and Mars, and gas giants like Jupiter and Saturn.\n4. The largest planet is Jupiter, which has a diameter of over 89,000 miles. The smallest planet is Mercury, which has a diameter of about 4,879 miles.\n5. Each planet has its own unique features, such as moons, rings, and atmospheres.", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-03-26T13:49:02.862737Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2610837125, 'load_duration': 31438708, 'prompt_eval_count': 42, 'prompt_eval_duration': 65890583, 'eval_count': 149, 'eval_duration': 2512801459, 'message': Message(role='assistant', content='', images=None, to

## StrOutput Parser

In [15]:
from langchain_core.output_parsers import StrOutputParser

In [17]:
str_chain= input_template | llm | StrOutputParser()
ans= str_chain.invoke({'topic':'Planets','points':'5','role':'teacher'})
print(ans)

Here are five key points about planets:

1. **There are eight planets in our solar system**: Mercury, Mars, Jupiter, Saturn, Uranus, Neptune, and Pluto.

2. **The four gas giants are massive**: Jupiter is the largest planet in our solar system, with a diameter of over 139,000 km.

3. **Mars is known as the Red Planet**: It has reddish appearance due to iron oxide in its soil.

4. **Jupiter's atmosphere is unique**: The planet has storm systems that can be thousands of kilometers wide and have winds up to 644 km/h.

5. **Pluto is no longer considered a planet**: In 2006, it was reclassified as a dwarf planet by the International Astronomical Union (IAU).


## Chain Multiple Runnables

In [18]:
str_chain

ChatPromptTemplate(input_variables=['points', 'role', 'topic'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['role'], input_types={}, partial_variables={}, template='You are {role}.You answer in short sentences'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['points', 'topic'], input_types={}, partial_variables={}, template='Tell me about {topic} in {points} points'), additional_kwargs={})])
| ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434')
| StrOutputParser()

In [19]:
analysis_prompt= ChatPromptTemplate.from_template(''' Analyze the following text : {prev_response}
                                                 You need to tell me how difficult it is to understand.
                                                  Answer in one sentence only
                                                  ''')

fact_check_chain = analysis_prompt | llm | StrOutputParser()
output = fact_check_chain.invoke({'prev_response': ans})
print(output)

The text requires a moderate level of understanding, with complex sentences and concepts such as "gas giants", unique features like Jupiter's storm systems, and the reclassification of Pluto.


In [27]:
composed_chain = {"prev_response": str_chain } | analysis_prompt | llm | StrOutputParser()
multiple_chain_output =  composed_chain.invoke({'topic':'Planets','points':'5','role':'phd teacher'})
print(multiple_chain_output)

The text requires a basic understanding of astronomy and the terms used to describe planets, with some minor explanations necessary for comprehension.


In [28]:
composed_chain = {"prev_response": str_chain } | analysis_prompt | llm | StrOutputParser()
multiple_chain_output =  composed_chain.invoke({'topic':'Planets','points':'5','role':'scientist'})
print(multiple_chain_output)

The text is written at a very basic level, using simple sentences and concepts that would be easy for an 8-year-old to understand.


## Parallel LCEL

In [30]:
question = HumanMessagePromptTemplate.from_template("Tell me about {topic} in {points} points")
sys_message = SystemMessagePromptTemplate.from_template("You are {role}.You answer in short sentences")

message=[sys_message,question]
template= ChatPromptTemplate(message)

fact_chain= template | llm | StrOutputParser()

fact_chain_output = fact_chain.invoke({'topic':'Planets','points':'5','role':'teacher'})
print(fact_chain_output)


Here's the information you requested:

1. **What are planets?** - Planets are massive, rocky or gaseous bodies that orbit around stars.

2. **7 planets in our solar system**: 
    * Mercury
    * Mars
    * Venus
    * Earth
    * Neptune
    * Uranus
    * Saturn

3. **Diameter and Mass of Planets**:
    - Mercury's diameter: 4,879 km
    - Mars' diameter: 6,794 km
    - Earth's diameter: 12,742 km
    - Neptune's diameter: 49,528 km
    - Uranus' diameter: 51,118 km

4. **Types of Planets**: 
    * Rocky planets (Mercury to Mars)
    * Gas giants (Jupiter, Saturn)

5. **How are planets discovered?**:
    - Telescopes can detect objects orbiting stars.
    - Astronomers study the movement and behavior of these objects.


In [31]:
question = HumanMessagePromptTemplate.from_template("Write poem on {topic} in {sentences} lines")


message=[sys_message,question]
template= ChatPromptTemplate(message)

poem_chain= template | llm | StrOutputParser()

poem_chain_output = poem_chain.invoke({'topic':'Planets','sentences':'5','role':'teacher'})
print(poem_chain_output)

Here's a short poem about planets:

Eight planets dance in space
Mercury's the closest, a tiny pace
Venus shines bright with heat and fire
Mars is red, with valleys high
Jupiter's massive, a gas so wide


In [32]:
from langchain_core.runnables import RunnableParallel

In [33]:
parallel_chain = RunnableParallel(fact=fact_chain,poem=poem_chain)
parallel_chain

{
  fact: ChatPromptTemplate(input_variables=['points', 'role', 'topic'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['role'], input_types={}, partial_variables={}, template='You are {role}.You answer in short sentences'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['points', 'topic'], input_types={}, partial_variables={}, template='Tell me about {topic} in {points} points'), additional_kwargs={})])
        | ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434')
        | StrOutputParser(),
  poem: ChatPromptTemplate(input_variables=['role', 'sentences', 'topic'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['role'], input_types={}, partial_variables={}, template='You are {role}.You answer in short sentences'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_va

In [35]:
parallel_chain_output = parallel_chain.invoke({'topic':'Planets','sentences':'5','points' : '8', 'role':'teacher'})
print(parallel_chain_output)

{'fact': 'Here are 8 key points about planets:\n\n1. Mercury is the closest planet to the sun, with temperatures ranging from -170°C to 427°C.\n2. Venus has a thick atmosphere that traps heat, making it the hottest planet at 462°C.\n3. Mars is known as the Red Planet due to its reddish appearance, which is caused by iron oxide in the soil.\n4. Jupiter is the largest planet, with a diameter of over 142,000 km and a mass more than 200 times that of Earth.\n5. Saturn has at least 62 moons, including the famous moon of Titan, which has a thick atmosphere.\n6. Uranus spins on its side, resulting in extreme seasons on this icy planet.\n7. Neptune is the farthest planet from the sun, with winds reaching up to 2,100 km/h.\n8. Pluto was previously considered a planet but is now classified as a dwarf planet due to its small size and irregular orbit.', 'poem': "Here's a short poem about planets:\n\nIn the sky, they orbit round,\nEight planets spinning, all around.\nMercury's first, closest to the

In [36]:
print(parallel_chain_output['fact'])
print('\n\n')
print(parallel_chain_output['poem'])

Here are 8 key points about planets:

1. Mercury is the closest planet to the sun, with temperatures ranging from -170°C to 427°C.
2. Venus has a thick atmosphere that traps heat, making it the hottest planet at 462°C.
3. Mars is known as the Red Planet due to its reddish appearance, which is caused by iron oxide in the soil.
4. Jupiter is the largest planet, with a diameter of over 142,000 km and a mass more than 200 times that of Earth.
5. Saturn has at least 62 moons, including the famous moon of Titan, which has a thick atmosphere.
6. Uranus spins on its side, resulting in extreme seasons on this icy planet.
7. Neptune is the farthest planet from the sun, with winds reaching up to 2,100 km/h.
8. Pluto was previously considered a planet but is now classified as a dwarf planet due to its small size and irregular orbit.



Here's a short poem about planets:

In the sky, they orbit round,
Eight planets spinning, all around.
Mercury's first, closest to the sun,
Venus shines bright, with

## Chain Router

In [42]:
prompt = """ Given the user review below, classify it as either 'Positive' or 'Negative'
             Do not respond with more than one word.

             Review : {review}
             Classification : """

template = ChatPromptTemplate.from_template(prompt)
chain = template | llm | StrOutputParser()

review = "Thank you so much for providing such a great service . I am really happy with the service"
chain.invoke({'review': review})

'Positive'

In [41]:
review = "I am not happy with the service. It is not good"
chain.invoke({'review': review})

'Negative'

In [47]:
Positive_prompt = """ 
                You are an expert in writing replies for positive reviews.
                You need to encourage the user to share their experience on social media.

                Review : {review}
                Answer : """


Positive_template = ChatPromptTemplate.from_template(Positive_prompt)
Positive_chain = Positive_template | llm | StrOutputParser()

In [44]:
Negative_prompt = """ 
                You are an expert in writing replies for negative reviews.
                You need to first apologise for the inconvenience caused to the user.
                You need to encourage the user to share their concern on following email 'abc.com'.

                Review : {review}
                Answer : """


Negative_template = ChatPromptTemplate.from_template(Negative_prompt)
Negative_chain = Negative_template | llm | StrOutputParser()

In [48]:
def route(info):
    if 'positive' in info['sentiment'].lower():
        return Positive_chain
    else:
        return Negative_chain

    

In [49]:
route({'sentiment':'positive'})

ChatPromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, template=' \n                You are an expert in writing replies for positive reviews.\n                You need to encourage the user to share their experience on social media.\n\n                Review : {review}\n                Answer : '), additional_kwargs={})])
| ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434')
| StrOutputParser()

## Runnable lambda

In [50]:
from langchain_core.runnables import RunnableLambda

In [53]:
full_chain = {'sentiment': chain , 'review' : lambda x: x['review']} | RunnableLambda(route)
full_chain

{
  sentiment: ChatPromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, template=" Given the user review below, classify it as either 'Positive' or 'Negative'\n             Do not respond with more than one word.\n\n             Review : {review}\n             Classification : "), additional_kwargs={})])
             | ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434')
             | StrOutputParser(),
  review: RunnableLambda(lambda x: x['review'])
}
| RunnableLambda(route)

In [56]:
review = "Thank you so much for providing such a great service . I am really happy with the service"
output= full_chain.invoke({'review': review})
print(output)

"I just wanted to take a moment to express my sincere gratitude for the amazing experience I had with your company. You truly delivered on your promises, and I'm thrilled with the results. If you're in need of any further assistance or simply want to share some great news, don't hesitate to reach out. I'd love to see what other wonderful experiences you have lined up! Share a post about your fantastic service on social media and let's spread the word!"


In [55]:
review = "I am not happy with the service. It is not good"
output= full_chain.invoke({'review': review})
print(output)

I'm so sorry to hear that your experience with our service hasn't met your expectations. We apologize for any inconvenience this has caused, and we understand that you're dissatisfied with the outcome.

We value your feedback and would like to help resolve this issue further. Could you please tell us more about what specifically didn't meet your expectations? Your input will assist us in improving our services to better serve all our customers.

Additionally, if there's anything on our website or in our recent communications that might have been missed or unclear, please let us know and we'll make sure to address it. We're committed to providing the best possible service to each of you, and your feedback is crucial in helping us achieve this goal.

If you ever need assistance or have any further concerns, don't hesitate to reach out to us directly through our website or by sending an email on abc.com. Your satisfaction is our priority, and we're always here to listen and respond. Thank

## Custom Chain Runnable

In [57]:
from langchain_core.runnables import RunnablePassthrough , RunnableLambda

In [67]:
def char_count(text):
    return len(text)

def word_count(text):
    return len(text.split())

In [64]:
prompt = ChatPromptTemplate.from_template("Explain these inputs in 4 sentences: {input1} and {input2}")

In [65]:
chain= prompt | llm | StrOutputParser()

output = chain.invoke({'input1': 'Earth is planet' , 'input2' : 'Sun is star'})
print(output)

You're referring to the basic elements of our solar system. The Earth is a planet, specifically one of the rocky planets located in the outer reaches of the solar system. A star, on the other hand, is a massive celestial body that generates its own light and heat through nuclear reactions in its core, such as the Sun. This fundamental distinction between planets and stars helps us understand the unique characteristics and properties of each type of object in our cosmic neighborhood.


In [70]:
runnable_chain = prompt | llm | StrOutputParser() | {'char_count': RunnableLambda(char_count),
                                                      'word_count' : RunnableLambda(word_count),
                                                      'output': RunnablePassthrough()}
runnable_output = runnable_chain.invoke({'input1': 'Earth is planet' , 'input2' : 'Sun is star'})
print(runnable_output)

{'char_count': 486, 'word_count': 84, 'output': 'You are referring to the basic components of our solar system. The Earth is a planet, specifically a terrestrial planet that orbits around the Sun due to its massive size and gravitational pull. In this context, a star is simply an enormous ball of hot, glowing gas, such as our Sun, which shines light and heat into space. When we say "planet" or "star", these terms are often used interchangeably, but technically speaking, planets are larger and more massive than stars like the Sun.'}


## Chain Decoractor


In [71]:
from langchain_core.runnables import chain

In [72]:
@chain

def custom_chain(params):
    return{
        'fact': fact_chain.invoke(params),
        'poem': poem_chain.invoke(params)
    }

params= {'topic':'Planets','sentences':'3','points' : '2', 'role':'teacher'}
output = custom_chain.invoke(params)

print(output['fact'])
print('\n\n')
print(output['poem'])

Planets are massive, rocky or gaseous bodies that orbit around a star. Here are two key points:

1. There are eight planets in our solar system: Mercury, Mars, Venus, Earth, Neptune, Uranus, Saturn, and Jupiter.
2. Planets have unique features like atmosphere, moons, rings, and magnetic fields that make them interesting to study.



Planets spinning in the sky,
Eight celestial bodies passing by,
Dancing stars, a celestial sigh.
